In [1]:
import pandas as pd
from os.path import join, isdir

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from icecream import ic
from src.utils import variant_utils as vu
from glob import glob
from itertools import product
from collections import defaultdict
from tqdm.notebook import tqdm
import pickle

from mplh.color_utils import create_color_df_v02
from mplh import cluster_help as ch

from src.utils import dendro_clust as dc

fig_utils from mplh


In [2]:
import sys
sys.setrecursionlimit(100000)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/mt_as_clones/variants_simpleUnion/dendro/af.0.1_othaf.0.1_cov.10_othcov.10_ncells.10_othncells.0.25_mean.0/"
# outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_simpleUnion/mt_clones_thresh/dendro/"
# N_DONORS=2

indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/mt_as_clones/variants_init/bestparams/af.0.1_othaf.0.1_cov.10_othcov.10_ncells.10_othncells.0.25_mean.0" 
outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/mt_as_clones/variants_init/dendro/af.0.1_othaf.0.1_cov.10_othcov.10_ncells.10_othncells.0.25_mean.0"
N_DONORS = 2


##se_cells_meta_f = /data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simpleUnion/knn/kparam_50/gff_A2_black/annotation_clones/se_cells_meta_labels.tsv 
#outdir = /data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/variants_simpleUnion/clonal_shifts/dendro_bc/results/inputOnly/knn/kparam_50/dendro_dt_0.6 

In [ ]:
cells_dir=join(indir, "cells")

In [ ]:
#samples = samples.split(",")

### Smaller params

In [ ]:
def fill_mt_bin(curr_pos, curr_cells):
    print(curr_pos.name)
    curr_pos.loc[curr_cells[curr_pos.name]["other_cells"]] = 0
    curr_pos.loc[curr_cells[curr_pos.name]["clone_cells"]] = 1
    return curr_pos

In [ ]:
bin_d = {}
#best_params = pd.read_csv(join(indir,  "best_params.csv"))
#print('params', best_params)
for d in range(N_DONORS): 
    bin_d[d] = pd.read_csv(join(indir,  f"donor_{d}_binary.csv"), index_col=0)


### Remove cells with all 0s and vars with less than min_cell number of cells
for d in bin_d:
    print(d)
    print(bin_d[d].shape)
    bin_d[d] = bin_d[d].loc[~((bin_d[d])==0).all(axis=1),~((bin_d[d])==0).all(axis=0)]
    print(bin_d[d].shape)

In [ ]:
%matplotlib inline

for d in bin_d:
    print(f"Donor {d}")    
    #df = bin_d[d].fillna(0).astype(bool).sample(1000)
    df = bin_d[d].fillna(0).astype(bool)
    
    # Drop the all 0s
    print(f"before 0 remove: {df.shape}")
    df = df.loc[~((df == 0).all(axis=1))]
    curr_clones = pd.DataFrame(index=df.index)
    curr_clones["Donor"] = d
    print(f"after 0 remove: {df.shape}")    
    den, curr_clones = dc.run_dendro_clust(df, dendroThresh=0.6, curr_clones=curr_clones, out_f=None,
                                           to_clust_stats=False, use_seaborn=False, verbose=True)
    
#     g = sns.clustermap(df, row_cluster=True, col_cluster=True, metric="jaccard")
#     inds = g.dendrogram_row.dendrogram["leaves"]
#     cols = g.dendrogram_col.dendrogram["leaves"]
#     curr_clones = curr_clones.iloc[inds]
#     plt.close(g.fig)

#     den = dc.dendro_cluster(df, g, d_thresh=0.8, no_labels=True, truncate_mode= 'lastp')
#     curr_clones = dc.add_cluster_labels(den, curr_clones)
#     curr_clones

    curr_clones["condition"]= [x.split("_")[-1] for x in curr_clones.index]
    curr_clones = curr_clones.loc[df.index]
    curr_clones

    f = plt.figure()
    sns.countplot(curr_clones["den_clust"])
    curr_clones["den_clust"] = curr_clones["Donor"].astype(str) +  "_" + curr_clones["den_clust"]
    #may want to sample when plotting
    ch.plot_cluster(df.fillna(0), row_meta=curr_clones, metric="jaccard", row_clr_schemes="categorical")
    plt.savefig(join(outdir, f"don_{d}_dendro_mt_as_clones.png"))
    curr_clones.to_csv(join(outdir, f"don_{d}_mt_dendro_clust.csv"))

In [ ]:
curr_clones

In [ ]:
outdir

In [ ]:
#ch.plot_cluster(df, row_meta=curr_clones, row_clr_schemes="categorical")

In [ ]:
# %matplotlib inline
# df = bin_d[d].fillna(0).astype(bool).sample(1000)
# curr_clones = pd.DataFrame(index=df.index)
# curr_clones["Donor"] = d
# g = sns.clustermap(df, row_cluster=True, col_cluster=True, metric="jaccard")
# inds = g.dendrogram_row.dendrogram["leaves"]
# cols = g.dendrogram_col.dendrogram["leaves"]
# curr_clones = curr_clones.iloc[inds]
# plt.close(g.fig)

# den_high = dc.dendro_cluster(df, g, d_thresh=0.6, no_labels=True, truncate_mode= 'lastp')

# curr_clones = dc.add_cluster_labels(den_high, curr_clones)
# curr_clones

# #size_pvals = dc.cluster_stats(curr_clones, out_f= None )

# curr_clones["condition"]= [x.split("_")[-1] for x in curr_clones.index]
# curr_clones = curr_clones.loc[df.index]
# curr_clones

# f = plt.figure()
# sns.countplot(curr_clones["den_clust"])

# curr_clones["den_clust"] = curr_clones["Donor"].astype(str) +  curr_clones["den_clust"]